In [37]:
import matplotlib.pyplot as plt
import threading
import pickle as pkl
import pandas as pd
import numpy as np
import cv2
import os
import re

<h1 style='text-align:center'>Loading Models</h1>

In [38]:
ols_model_file_path = '../task1/Linear_Reg_Images_my_data.sav'
sgd_model_file_path = '../task2/SGF_Linear_Reg_Images_my_data.sav'
identity_model_file_path = '../../Assignment 1/task3/Decision_model_face_identity.sav'

In [39]:
ols_model = pkl.load(open(ols_model_file_path,'rb'))
sgd_model = pkl.load(open(sgd_model_file_path,'rb'))
identity_model = pkl.load(open(identity_model_file_path,'rb'))

In [40]:
models = [ols_model,sgd_model,identity_model]

<h1 style='text-align:center'>Webcam app</h1>

<h3 style='text-align:center'>Preprocessing</h3>

In [41]:
def Compute_histogram(img):
   blue_color = cv2.calcHist([img], [0], None, [256], [0, 256])
   red_color = cv2.calcHist([img], [1], None, [256], [0, 256])
   green_color = cv2.calcHist([img], [2], None, [256], [0, 256])
   list = blue_color.flatten() 
   list = np.concatenate((list, red_color.flatten()))
   list = np.concatenate((list, green_color.flatten()))
   
   return list

In [42]:
def Compute_3x3_vector(img):
    height,width = img.shape[0],img.shape[1]
    row1 = round(height/3)
    row2 = round(height*2/3)
    row3 = height-row2
    col1 = round(width/3)
    col2 = round(width*2/3)
    col3 = width-col2

    img1 = img[:row1,:col1]
    feature_vector = Compute_histogram(img1)
    img12 = img[:row1,col1:col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img12)))
    img13 = img[:row1,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img13)))

    img2 = img[row1:row2,:col1]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img2)))
    img22 = img[row1:row2,col1:col2]
    feature_vector = np.concatenate((feature_vector,Compute_histogram(img22)))

    img23 = img[row1:row2,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img23)))


    img3 = img[row2:row3+row2,:col1]
    feature_vector = np.concatenate((feature_vector,   Compute_histogram(img3)))
    img32 = img[row2:row3+row2,col1:col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img32)))

    img33 = img[row2:row3+row2,col2:col3+col2]
    feature_vector = np.concatenate((feature_vector, Compute_histogram(img33)))

    return feature_vector

In [43]:
def Compute_Feature_vector(img):
    img = cv2.resize(img, (224,224))
    frame = np.array([Compute_3x3_vector(img)])
    frame = Compute_Feature_vector(frame)
    age_ols = models[0].predict(frame)
    age_sgd = models[1].predict(frame)
    identity = models[2].predict(frame)
    print(age_ols,age_sgd,identity)
    cv2.putText(frame,str(age_ols),(0,221),cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame,str(age_sgd),(0,221),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame,str(identity),(0,221),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('Frame', frame)

In [44]:
def App(models):
    try:
        vidcap = cv2.VideoCapture(0)
        font = cv2.FONT_HERSHEY_SIMPLEX
        thickness = 2
        font_scale = 1
        if vidcap.isOpened():
            while True:
                ret, frame = vidcap.read()
                if ret:
                    threading.Thread(target=Compute_Feature_vector,args=(frame,)).start()
                    cv2.waitKey(0)
                    break
                    # frame = Compute_Feature_vector(frame)
                    # age_ols = models[0].predict(frame)
                    # age_sgd = models[1].predict(frame)
                    # identity = models[2].predict(frame)
                    # print(age_ols)
                    # cv2.putText(frame,str(age_ols),(0,221),font,font_scale, (255, 0, 0), thickness, cv2.LINE_AA)
                    # cv2.putText(frame,str(age_sgd),(0,221),font,font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
                    # cv2.putText(frame,str(identity),(0,221),font,font_scale, (0, 0, 255), thickness, cv2.LINE_AA)
                    # cv2.imshow('Frame', frame)
                else:
                    print("Frame not captured")
                if cv2.waitKey(5) & 0xFF==ord('q'):
                    break
        else:
            print("Video is not working")
        vidcap.release()
        cv2.destroyAllWindows() 
    except Exception as e:
        print(e)

In [45]:
App(models)

Exception in thread Thread-996:
Traceback (most recent call last):
  File "c:\Python\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Python\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hazim\AppData\Local\Temp\ipykernel_14032\3571656424.py", line 4, in Compute_Feature_vector
  File "C:\Users\Hazim\AppData\Local\Temp\ipykernel_14032\3571656424.py", line 3, in Compute_Feature_vector
  File "C:\Users\Hazim\AppData\Local\Temp\ipykernel_14032\3404846930.py", line 11, in Compute_3x3_vector
  File "C:\Users\Hazim\AppData\Local\Temp\ipykernel_14032\3294659080.py", line 3, in Compute_histogram
cv2.error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\histogram.cpp:160: error: (-215:Assertion failed) j < nimages in function 'cv::histPrepareImages'

